In [1]:
from transformers import AutoTokenizer
import transformers
import torch
import accelerate
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")
secret_value_1 = user_secrets.get_secret("NGROK_KEY")
model_chat = "meta-llama/Llama-2-13b-chat-hf"
tokenizer_chat = AutoTokenizer.from_pretrained(model_chat, token=secret_value_0)
pipeline_chat = transformers.pipeline(
    "text-generation",
    model=model_chat,
    torch_dtype=torch.float16,
    device_map="auto",
    token=secret_value_0,
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

2024-03-09 04:35:57.804181: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-09 04:35:57.804285: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-09 04:35:58.089480: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [2]:
!pip install langchain
from langchain import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipeline_chat)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 747.0 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 5.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are insta

In [3]:
from langchain import PromptTemplate
multi_input_template = """
<s>[INST] <<SYS>>
You are a helpful assistant that helps users generate questions. You will receive one question, and the question will be in JSON format. It will contain the question stem itself, no more than 4 options to answer the question, the correct option for the particular question, and the subject or topic related to the question.
You will do your best to generate one question, which should be similar in topic, style and difficulty of the question provided to you. It should be in JSON Format, and provide the question stem itself, the options, and the correct option to select.
You should ONLY have the JSON dictionary in your response, NOTHING else.
<</SYS>>

{sample_in} [/INST] {sample_out} </s><s>[INST] {query} [/INST]
"""

multi_input_prompt = PromptTemplate(input_variables=["sample_in", "sample_out", "query"], template=multi_input_template)

In [4]:
from langchain import LLMChain
llm_chain = LLMChain(prompt=multi_input_prompt, llm=llm)

In [5]:
!pip install tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.3 MB/s eta 0:00:00a 0:00:01


In [6]:
from tavily import TavilyClient
secret_value_3 = user_secrets.get_secret("TAVILY_API_KEY")
tavily_cli = TavilyClient(api_key=secret_value_3)

In [7]:
question_template = """
<s>[INST] <<SYS>>
You are a helpful assistant that helps label correct answers for question. You will receive one question, and the question will be in JSON format. It will contain the question stem itself, no more than 4 options to answer the question, and information to help you answer the question.
You will do your best to match the correct option. Your answer should be in JSON format, providing the correct answer. If the information provided states that the correct answer is something, not in the options, change one of the options to accomodate for the correct option.
You should ONLY have the JSON dictionary in your response, NOTHING else. The answer you provide should only be equal to A, B, C, D nothing else!
<</SYS>>

{sample_in} [/INST] {sample_out} </s><s>[INST] {query} [/INST]
"""

question_prompt = PromptTemplate(input_variables=["sample_in", "sample_out", "query"], template=question_template)

In [8]:
ques_chain = LLMChain(prompt=multi_input_prompt, llm=llm)

In [9]:
!pip install flask
!pip install pyngrok
!pip install flask-cors
!pip install redis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 1.8 MB/s eta 0:00:00a 0:00:01


In [10]:
import redis
secret_value_4 = user_secrets.get_secret("UPSTASH_PD")
redis_client = redis.StrictRedis(host="usw1-organic-yak-33107.upstash.io", port='33107', password=secret_value_4, decode_responses=True)

In [11]:
try:
    ngrok_json = redis_client.get('ngrok_link')
    if ngrok_json is not None:
        link = ngrok_json
    else:
        link = ""
except Exception:
    link = ""

In [12]:
sample_in="{'topic': 'Evolution', 'question': 'Man's evolution is defined as:', 'options': ['Dysversion', 'Anthropogenesis', 'Convergence', 'ontogenesis'], 'answer': 'B'}"
sample_out="{'question': 'Which of the following mechanisms is NOT a driving force of evolution?', 'options': ['Natural selection', 'Genetic drift', 'Lamarckism', 'Geographical barriers'], 'answer': 'C'}"
ques_in="{'topic': 'Kinematics', 'question': 'Which of the following values is a vector?', 'options': ['Speed', 'Distance', 'Position', 'Velocity'], 'info': 'D. Velocity is a vector, as it has a direction'}"
ques_out="{'topic': 'Kinematics', 'question': 'Which of the following values is a vector?', 'options': ['Speed', 'Distance', 'Position', 'Velocity'], 'info': 'D. Velocity is a vector, as it has a direction', 'answer': 'D'}"

In [18]:
from flask import Flask, jsonify, request
import threading
from pyngrok import ngrok
import json
from flask_cors import CORS
import re
import time
app = Flask(__name__)
CORS(app)
@app.route('/run', methods=['GET'])
def hello_world():
    try:
        data_in = request.args.get('msg')
        print(data_in)
        start = time.time()
        data_out = llm_chain.invoke({"sample_in": sample_in, "sample_out": sample_out, "query": data_in})['text']
        print(data_out)
        print(time.time()-start)
        pattern = r"(?<=\W)'(.*?)'(?=\W)"
        output_string = re.sub(pattern, lambda x: f'"{x.group(1)}"', data_out)
        dict_ans = json.loads(output_string)
        query_tavily = dict_ans["question"] + ": "
        opt_str = "ABCD"
        for n in range(4):
            query_tavily += opt_str[n] + ". " + dict_ans["options"][n] + "  "
        context = tavily_cli.qna_search(query=query_tavily[:-2])
        print(context)
        try:
            del dict_ans["answer"]
        except Exception as e:
            pass
        dict_ans["info"] = context
        final_response = ques_chain.invoke({"sample_in": ques_in, "sample_out": ques_out, "query": dict_ans})["text"]
        print(final_response)
        return jsonify({"output": final_response})
    except Exception as e:
        print(e)

def run_flask():
    app.run(port=1000)

ngrok_auth_token = secret_value_1
ngrok.set_auth_token(ngrok_auth_token)
public_url = ngrok.connect(1000)
threading.Thread(target=run_flask).start()
pattern_link = r"https?://\S+"
link = str(re.findall(pattern_link, str(public_url))[0])[:-1]
redis_client.set('ngrok_link', link+"/")
print(f"Access your Flask app at: {public_url}")

 * Serving Flask app '__main__'
 * Debug mode: off
Access your Flask app at: NgrokTunnel: "https://c4f8-34-70-127-179.ngrok-free.app" -> "http://localhost:1000"
{"topic":"Chemistry","question":"What's the chemical formula of Water","options":["H2O","OH2","O2H","HO2"],"answer":"A"}


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



{'question': 'What is the chemical formula of hydrogen peroxide?', 'options': ['H2O2', 'H2O', 'O2H', 'HO2'], 'answer': 'B'}
5.714832305908203
A. H2O2


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{"topic":"Chemistry","question":"What's the chemical formula of Water","options":["H2O","OH2","O2H","HO2"],"answer":"A"}


...


{'question': 'Which of the following elements is NOT a member of the noble gas family?', 'options': ['Nitrogen', 'Oxygen', 'Helium', 'Fluorine'], 'info': 'The element NOT a member of the noble gas family is Fluorine', 'answer': 'B'}
{"topic":"Chemistry","question":"What's the chemical formula of Water?","options":["H2O","OH2","O2H","HO2"],"answer":"A"}


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



{'question': 'What is the chemical formula of hydrochloric acid?', 'options': ['HClO4', 'H2O', 'HCl', 'HOCl'], 'answer': 'B'}
5.595081806182861
C. HCl


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



{'question': 'What is the chemical formula of hydrochloric acid?', 'options': ['HClO4', 'H2O', 'HCl', 'HOCl'], 'info': 'C. HCl', 'answer': 'C'}
